# Superoperator checks

In [1]:
from itertools import product
import matplotlib.pyplot as plt

import time
import numpy as np
#import cmath
import pysqkit
#from pysqkit.solvers import solvkit
from typing import List
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import qutip as qtp

import pysqkit.util.transformations as trf
from pysqkit.util.linalg import hilbert_schmidt

#from pysqkit.util.phys import temperature_to_thermalenergy
#import pysqkit.util.transformations as transf
#from pysqkit.util.basis import pauli_basis
#from pysqkit.util.linalg import hilbert_schmidt
#from pysqkit.util.metrics import avg_process_fid
from IPython.display import display, Latex
#from pysqkit.util.linalg import get_mat_elem, tensor_prod

In [2]:
cz_qobj = qtp.qobj.Qobj(np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]]))
cz_super = qtp.to_super(cz_qobj)
rho_zero = (qtp.qeye(2) + qtp.sigmaz())/2
rho_one = (qtp.qeye(2) - qtp.sigmaz())/2
cz_tensor = qtp.tensor(rho_zero, qtp.qeye(2)) + qtp.tensor(rho_one, qtp.sigmaz())
cz_tensor_super = qtp.to_super(cz_tensor)

I just check that the two ways of getting the CZ superoperator, which differ only by the tensor product structure in Qutip give the same result.

In [3]:
np.diag(cz_super[:, :])

array([ 1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j,
       -1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j,
       -1.+0.j,  1.+0.j])

In [4]:
np.diag(cz_tensor_super[:, :])

array([ 1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j,
       -1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j,
       -1.+0.j,  1.+0.j])

## I now check the TomoEnv class
We do checks for a fluxonium transmon system

In [5]:
levels_f = 5
levels_t = 3
d_comp = 4
d_leak = levels_t*levels_f - d_comp
flx = pysqkit.qubits.Fluxonium(label='F', charge_energy=.973, induct_energy=.457, 
                               joseph_energy=8.0, flux=1/2, dim_hilbert=100)
trans = pysqkit.qubits.SimpleTransmon(label='T', max_freq=4.5, 
                                      anharm=-0.3, dim_hilbert=levels_t)

flx.diagonalize_basis(levels_f)
energies_f, eig_states_f = flx.eig_states(levels_f)
energies_t, eig_states_t = trans.eig_states(levels_t)  

In [6]:
#Drive
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

In [7]:
jc = 0.07
coupled_sys = trans.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
energies_sys, eigstates_sys = coupled_sys.eig_states() 

coupled_sys_bare = trans.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0)
energies_bare, eigstates_bare = coupled_sys_bare.eig_states()
    
energies_in_lexico = np.zeros(levels_f*levels_t, dtype=float)
count = 0
for k in range(0, levels_t):
    for m in range(0, levels_f):
        energies_in_lexico[count] = energies_t[k] + energies_f[m]
        count += 1
ascending_to_lexico = np.argsort(energies_in_lexico)
label_converter = ascending_to_lexico

In [8]:
def label_to_states(
    k: int, 
    m: int,
    levels_f: int,
    eig_states: np.ndarray,
    label_converter: np.ndarray 
) -> float:
    label = k*levels_f + m
    index = np.where(label_converter == label)[0][0] #index such that converter[index] = label
    return eig_states[index] 

In [9]:
eig_states_by_label = []
energies_by_label = []
for i_a in range(levels_t):
    energies_by_label.append([])
    eig_states_by_label.append([])
    for i_b in range(levels_f):
        eig_states_by_label[i_a].append(qtp.Qobj(inpt=  label_to_states(i_a, i_b, levels_f, eigstates_sys, label_converter), 
                                                 dims=[[levels_t, levels_f], [1, 1]], shape=[levels_t*levels_f, 1]))
        energies_by_label[i_a].append(label_to_states(i_a, i_b, levels_f, energies_sys, label_converter))

In [10]:
eps_f = 0.03091755984900732
freq_drive = 7.155
t_rise = 15 #16.0
t_tot = 60 # 60
pts_per_drive_period = 15

nb_points = int(t_tot*freq_drive*pts_per_drive_period)
tlist = np.linspace(0, t_tot, nb_points)

interest_levels = [[0,0], [0,1], [1,0], [1,1], [0,2], [0,3], [1,3], [2,0]]
labels = ["$| "+ str(k) + str(m) + " \\rangle $" for k,m in interest_levels]

hamil0 = coupled_sys.hamiltonian(as_qobj=True)
coupled_sys['F'].drives['cz_drive_f'].set_params(phase=0, time=tlist, rise_time=t_rise, 
                                                 amp=eps_f, freq=freq_drive)

In [11]:
drive_hamils = []
pulses = []

for qubit in coupled_sys:
    if qubit.is_driven:
        for label, drive in qubit.drives.items():
            drive_hamils.append(drive.hamiltonian(as_qobj=True))
            pulses.append(drive.eval_pulse())

In [12]:
temperature = 0.020 # K
en_th = pysqkit.util.phys.temperature_to_thermalenergy(temperature) # kb T/h in GHz
tan_deltac_f = 7.1*1e-6 
tan_deltac_t = 0.75*1e-6
qdiel_f = 1/tan_deltac_f
qdiel_t = 1/tan_deltac_t
trans_jumps = trans.dielectric_loss(qdiel_t, 1/en_th, as_qobj=True)
flx_jumps = flx.dielectric_loss(qdiel_f, 1/en_th, as_qobj=True)
jumps = []
for op in trans_jumps:
    jumps.append(qtp.tensor(op, qtp.qeye(levels_f)))
for op in flx_jumps:
    jumps.append(qtp.tensor(qtp.qeye(levels_t), op))

We introduce the TomoEnv object

In [13]:
env_syst = pysqkit.tomography.TomoEnv(system = coupled_sys, jump_op=jumps)

In [14]:
comp_states = [eig_states_by_label[0][0][:], eig_states_by_label[0][1][:], eig_states_by_label[1][0][:], \
               eig_states_by_label[1][1][:]]

In [15]:
start = time.time()
sup_op = env_syst.to_super(comp_states, trf.weyl_by_index)
end = time.time()
print("Computational time = " + str(end - start) + ' s')

Computational time = 60.89344382286072 s


Now we check that the probability of finding the system in one of the computational basis states given a certain initial state agrees with the corresponding calculation using the superoperator representation.

In [60]:
ket_in = eig_states_by_label[1][1]
rho_in = ket_in*ket_in.dag()
output = pysqkit.solvers.solvkit.integrate(tlist*2*np.pi, rho_in, hamil0, drive_hamils, pulses, jumps, "mesolve")
rho_fin = output.states[-1]

Using the superoperator

In [61]:
basis = [] 
d = len(comp_states)
for i in range(0, d**2):
            basis.append(trf.iso_basis(i, comp_states, trf.weyl_by_index))
rho_in_vec = np.zeros(d**2, dtype=complex)
for i in range(0, d**2):
    rho_in_vec[i] = hilbert_schmidt(basis[i], rho_in[:, :])
rho_fin_vec = sup_op.dot(rho_in_vec)
rho_fin_from_sup = 0
for i in range(0, d**2):
    rho_fin_from_sup += rho_fin_vec[i]*basis[i]

We now compute the fidelity between the state computed using the direct master equation simulation and the superoperator. I first remark that

In [65]:
qtp.fidelity(rho_fin, rho_fin)

1.000475331144416

The accuracy is clearly not satisfactory as the fidelity of the final state with itself is 1 only up to the 5th digit.

In [70]:
qtp.fidelity(rho_fin, qtp.Qobj(inpt=rho_fin_from_sup, dims=[[3, 5], [3, 5]]))

1.0005820967313594

Thus, the result is accurate given the the numerical accuracy. Also, notice that we expect the previous quantity to be 1 only when we have low leakage.